In [ ]:
import serial
import numpy as np
from PIL import Image
import re

IMAGE_WIDTH = 80
IMAGE_HEIGHT = 60

def clean_hex_string(hex_str):
    # Remove any non-hexadecimal characters
    return re.sub(r'[^0-9a-fA-F]', '', hex_str)

def hex_to_bytes(hex_str):
    cleaned_hex = clean_hex_string(hex_str)
    if len(cleaned_hex) % 2 != 0:
        cleaned_hex = cleaned_hex[:-1]  # Ensure even length
    return bytes.fromhex(cleaned_hex)

def receive_lora_data(serial_port='/dev/ttyUSB0', baud_rate=115200):
    ser = serial.Serial(serial_port, baud_rate, timeout=1)
    received_data = ''
    image_counter = 0

    try:
        while True:
            if ser.in_waiting:
                chunk = ser.read(ser.in_waiting).decode('ascii', errors='ignore')
                received_data += chunk

                while len(clean_hex_string(received_data)) >= IMAGE_WIDTH * IMAGE_HEIGHT * 2:
                    # Extract and clean 4800 bytes worth of hex data
                    hex_data = clean_hex_string(received_data)[:IMAGE_WIDTH * IMAGE_HEIGHT * 2]
                    received_data = received_data[received_data.index(hex_data[-1]) + 1:]

                    try:
                        # Convert hex to bytes
                        byte_data = hex_to_bytes(hex_data)

                        # Convert to uint8 and reshape into 2D array
                        image_array = np.frombuffer(byte_data, dtype=np.uint8)
                        image_array = image_array.reshape((IMAGE_HEIGHT, IMAGE_WIDTH)).T

                        print(f"Array min: {image_array.min()}, max: {image_array.max()}, mean: {image_array.mean():.2f}")

                        # Create and save image
                        pil_image = Image.fromarray(image_array, mode='L')
                        image_counter += 1
                        pil_image.save(f"/home/cody/Desktop/img/received_image_{image_counter}.png")
                        print(f"Saved image {image_counter}")

                    except Exception as e:
                        print(f"Error processing image: {e}")
                        print(f"Problematic hex data: {hex_data[:100]}...")  # Print first 100 chars for debugging

    except KeyboardInterrupt:
        print("Stopping data reception.")
    finally:
        ser.close()

if __name__ == "__main__":
    receive_lora_data()

Array min: 0, max: 255, mean: 126.82
Saved image 1
Array min: 0, max: 255, mean: 77.94
Saved image 2
Array min: 0, max: 255, mean: 77.94
Saved image 3
Array min: 0, max: 255, mean: 75.25
Saved image 4
Array min: 0, max: 255, mean: 58.48
Saved image 5
Array min: 0, max: 255, mean: 57.75
Saved image 6
Array min: 0, max: 250, mean: 124.67
Saved image 7
Array min: 0, max: 255, mean: 56.68
Saved image 8
